In [1]:
%matplotlib inline
import os, sys, gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.tri as tri
from collections import Counter
from scipy.special import factorial
import itertools
from math import comb
from scipy import stats

In [2]:
version = '1.3'
load_dir = '../data/human/{}'.format(version)
save_dir = load_dir
print(os.listdir(load_dir))

['html-button-response_processed_1.3.csv', 'processed_data_set_level_1.3.p', 'html-button-response_processed.csv', 'participants.csv', 'processed_data_exp_level_1.3.p', 'html-button-response.csv', 'exclusion_data_1.3.csv', 'survey-multi-select.csv', 'html-keyboard-response.csv', 'response_distribution.png', 'demographics.numbers', 'processed_data_1.3.p', 'demographics.csv']


# Data analysis
In this script, we apply classical and Bayesian approaches to determine whether the data provide significant evidence of deviations from independence between features and relations. We will either focus on the strong or weak MAX effect, or MAX and MIN effects together (both of these represent deviations). 

The data are presented below. (D-B) - (C-A) positive represents weak support for MAX, and Max indicates whether strong support was also found. Strong support for a participant implies weak support. 

In [3]:
fullDataDF = pd.read_pickle('{}/processed_data_exp_level_{}.p'.format(save_dir, version)).to_frame()
setDataDF = pd.read_pickle('{}/processed_data_set_level_{}.p'.format(save_dir, 
                                                                     version)).transpose().set_axis(['Set {}'.format(x) for x in np.arange(1, 7)], axis=1, inplace=False)


fullDataDF.set_axis(['Aggregate Data'], axis=1, inplace=True)
setDataDF['Mean'] = setDataDF.mean(numeric_only=True, axis=1)

In [12]:
def return_statistic(alpha, beta, option="z", N=30):
    """Currently assumes two-tailed"""
    if option == "z":
        return (stats.norm.ppf(1-alpha/2), stats.norm.ppf(beta))
    elif option == "t":
        return (stats.t.ppf(1-alpha/2, N-1), stats.t.ppf(beta, N-1))
    else:
        print("Statistic not implemented yet")
        return 
    
def return_p(statistic, option="z", N=30):
    if option == "z":
        return stats.norm.cdf(statistic)
    elif option == "t":
        return stats.t.cdf(statistic, N-1)
    else:
        print("Statistic not implemented yet")
        return 
    
def stat_p_val_power(mu_hat, sigma_hat, N=None, alpha=0.05, mu_0=0, option="z", tails="both"):
    """Calculate P value and power of our sample"""  
    if option=="t":
        stat = (mu_hat - mu_0)/(sigma_hat/ np.sqrt(N-1))
        power = (mu_0 - mu_hat) / (sigma_hat / np.sqrt(N-1))
        p1 = return_p(power + stats.t.ppf(alpha/2, N-1), option=option, N=N) 
        p2 = 1 - return_p(power + stats.t.ppf(1-(alpha/2), N-1), option=option, N=N) 
    elif option=="z":
        stat = (mu_hat - mu_0)/(sigma_hat/ np.sqrt(N))
        power = (mu_0 - mu_hat) / (sigma_hat / np.sqrt(N))
        
        p1 = return_p(power + stats.norm.ppf(alpha/2), option=option, N=N) 
        p2 = 1 - return_p(power + stats.norm.ppf(1-(alpha/2)), option=option, N=N) 
    
    # if this is not true, would need to invert below
    if tails=="both" and stat >= 0:
        p_val = (1 - return_p(stat, option=option, N=N)) + return_p(-stat, option=option, N=N)
    elif tails=="both" and stat < 0: # I think the below works, but need to work through
        assert stat >= 0, "stat not greater than zero: {}".format(stat)
        p_val = (1 - return_p(-stat, option=option, N=N)) + return_p(stat, option=option, N=N)
    elif tails=="upper":
        p_val = (1 - return_p(stat, option=option, N=N)) 
    elif tails=="lower":
        p_val = return_p(stat, option=option, N=N)
    power = p1 + p2
    return (stat, p_val, power)

def temp_set_function(df, mu_name, var_name, tails="both", mean_col=True):
    results = []
    for set_num, data in df.iteritems():
        N_t = data.loc["N"]
        mu_t = data.loc[mu_name]
        sigma_t = np.sqrt(data.loc[var_name])
        t_t, p_t, _ = stat_p_val_power(mu_t, sigma_t, N=N_t, option="t", tails=tails) # post-hoc
        results.append([t_t, p_t, N_t-1])
        #print("All data significance is (t={}, p={}, dof={})".format(t_t, p_t, N_t-1))
    if mean_col:
        results.pop()
    return results

def binomial_probability(h, p, N):
    return comb(int(N), int(h)) * (p**h) * ((1-p)**(N-h))

def binomial_test(h, p, N_m):
    # check this
    p_val = 0
    for i in np.arange(h, N_m+1): # number of maxes "heads"
        p_val += binomial_probability(i, p, N_m)
    return p_val

In [13]:
display(fullDataDF.round(2))
N = fullDataDF.loc["N"].iloc[0]
mu = fullDataDF.loc["(D-B)-(C-A)"].iloc[0]
sigma = fullDataDF.loc["SD_all"].iloc[0]
print('Mu is {}, sigma is {}, N is {}'.format(mu, sigma, N))
t, p, _ = stat_p_val_power(mu, sigma, N=N, option="t", tails="both")

mu_C_A = fullDataDF.loc["C-A"].iloc[0]
sigma_C_A = np.sqrt(fullDataDF.loc["Var_C-A_all"].iloc[0])
print('Mu C-A is {}, sigma C-A is {}, N is {}'.format(mu_C_A, sigma_C_A, N))
t_C_A, p_C_A, _ = stat_p_val_power(mu_C_A, sigma_C_A, N=N, option="t", tails="lower")

mu_D_B = fullDataDF.loc["D-B"].iloc[0]
sigma_D_B = np.sqrt(fullDataDF.loc["Var_D-B_all"].iloc[0])
print('Mu D_B is {}, sigma D_B is {}, N is {}'.format(mu_D_B, sigma_D_B, N))
t_D_B, p_D_B, _ = stat_p_val_power(mu_D_B, sigma_D_B, N=N, option="t", tails="upper")

aggregateResults = pd.DataFrame([[t, p, N-1], [t_C_A, p_C_A, N-1], [t_D_B, p_D_B, N-1]])
aggregateResults.set_axis(['t', 'p', 'DOF'], axis=1, inplace=True)
aggregateResults.set_axis(['(D-B)-(C-A)', 'C-A', 'D-B'], axis=0, inplace=True)
display(aggregateResults.round(5))

print(binomial_test(fullDataDF.loc["N_max"][0], 0.5, fullDataDF.loc["N_max"][0]+fullDataDF.loc["N_min"][0]))

,Aggregate Data
A,5.01
B,3.30
C,4.54
D,3.35
C-A,-0.48
D-B,0.05
(D-B)-(C-A),0.52
N_max,21.00
N_min,7.00
N,960.00


Mu is 0.5229166666666667, sigma is 0.8652708110117254, N is 960.0
Mu C-A is -0.47552083333333334, sigma C-A is 1.0651354926118686, N is 960.0
Mu D_B is 0.047395833333333325, sigma D_B is 0.8681588653801765, N is 960.0


,t,p,DOF
(D-B)-(C-A),18.71500,0.00000,959.0
C-A,-13.82528,0.00000,959.0
D-B,1.69064,0.04562,959.0


0.006270475685596466


In [23]:
display(setDataDF.round(2))
# this gives the significance of main variable 
mu_name = "(D-B)-(C-A)"
var_name = "Var_set"
results = temp_set_function(setDataDF, mu_name, var_name)
sigRes = pd.DataFrame(results)
sigRes.set_axis(['t', 'p', 'DOF'], axis=1, inplace=True)
sigRes.set_axis(['Set {}'.format(x) for x in np.arange(1, 7)], axis=0, inplace=True)
display(sigRes.round(5))
print(setDataDF.loc["N_max"])
binTests = [binomial_test(setDataDF.loc["N_max"].loc["Set {}".format(x)], 0.5, setDataDF.loc["N_max"].loc["Set {}".format(x)] + setDataDF.loc["N_min"].loc["Set {}".format(x)])
                          for x in np.arange(1,7)]
binTestDF = pd.DataFrame(binTests)
binTestDF.set_axis(['Exact binomial test p value'], axis=1, inplace=True)
binTestDF.set_axis(['Set {}'.format(x) for x in np.arange(1, 7)], axis=0, inplace=True)
display(binTestDF)

,Set 1,Set 2,Set 3,Set 4,Set 5,Set 6,Mean
A,5.62,4.37,5.32,4.82,5.16,4.78,5.01
B,1.92,2.74,4.76,3.25,2.91,4.21,3.30
C,5.08,3.34,4.94,4.71,4.76,4.41,4.54
D,2.13,2.52,4.51,3.43,3.30,4.18,3.35
C-A,-0.54,-1.03,-0.39,-0.12,-0.40,-0.38,-0.48
D-B,0.21,-0.22,-0.25,0.18,0.39,-0.03,0.05
(D-B)-(C-A),0.76,0.81,0.14,0.29,0.79,0.35,0.52
N_max,1.00,5.00,3.00,2.00,7.00,3.00,3.50
N_min,0.00,1.00,1.00,3.00,0.00,2.00,1.17
N,160.00,160.00,160.00,160.00,160.00,160.00,160.00


,t,p,DOF
Set 1,5.66108,0.00000,159.0
Set 2,5.28896,0.00000,159.0
Set 3,0.96736,0.33483,159.0
Set 4,2.28030,0.02392,159.0
Set 5,5.03720,0.00000,159.0
Set 6,2.17847,0.03084,159.0


Set 1    1.0
Set 2    5.0
Set 3    3.0
Set 4    2.0
Set 5    7.0
Set 6    3.0
Mean     3.5
Name: N_max, dtype: float64


,Exact binomial test p value
Set 1,0.500000
Set 2,0.109375
Set 3,0.312500
Set 4,0.812500
Set 5,0.007812
Set 6,0.500000


In [ ]:
mu_names = ["C-A", "D-B"]
var_names = ["Var_C-A_set", "Var_D-B_set"]
tails = ["lower", "upper"]
allResults = []
for mu_name_t, var_name_t, tails_t in zip(mu_names, var_names, tails):
    allResults.append(pd.DataFrame(temp_set_function(setDataDF, mu_name_t, var_name_t, tails=tails_t)))
    display(allResults[-1])
allComps = pd.concat(allResults, axis=1)
allComps.set_axis(['Set {}'.format(x) for x in np.arange(1, 7)], axis=0, inplace=True)
allComps.set_axis(['t (C-A)', 'p (C-A)', 'DOF (C-A)', 't (D-B)', 'p (D-B)', 'DOF (D-B)'], axis=1 , inplace=True)
display(allComps)